In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"
import yfinance as yf

In [ ]:
# List of tickers to download
tickers = ["TCS.NS", "INFY.NS", "WIPRO.NS", "HCLTECH.NS"]

# Download data for each ticker
data = yf.download(tickers, start="2023-01-01", end="2024-10-31", group_by="ticker")

# Step 1: Stack the DataFrame to get a long-format DataFrame
# This moves the ticker from columns to rows and creates a multi-level index with (Date, Ticker)
data = data.stack(level=0).reset_index()

# Step 2: Rename columns for readability
data.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Display the cleaned DataFrame
print(data.info())

[*********************100%***********************]  4 of 4 completed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   Date       1800 non-null   datetime64[ns, UTC]
 1   Ticker     1800 non-null   object             
 2   Open       1800 non-null   float64            
 3   High       1800 non-null   float64            
 4   Low        1800 non-null   float64            
 5   Close      1800 non-null   float64            
 6   Adj Close  1800 non-null   float64            
 7   Volume     1800 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 112.6+ KB
None



<ipython-input-25-5709cd4d92ac>:9: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0).reset_index()


In [ ]:
data.head(10)

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,2023-01-02 00:00:00+00:00,HCLTECH.NS,1039.099976,1042.500000,1031.500000,1039.449951,960.256714,701880
1,2023-01-02 00:00:00+00:00,INFY.NS,1514.000000,1526.500000,1504.050049,1524.000000,1459.632446,3801140
2,2023-01-02 00:00:00+00:00,TCS.NS,3261.000000,3266.850098,3235.199951,3261.449951,3160.044922,709547
3,2023-01-02 00:00:00+00:00,WIPRO.NS,393.000000,393.899994,390.500000,393.049988,391.249847,2779756
4,2023-01-03 00:00:00+00:00,HCLTECH.NS,1039.000000,1045.800049,1033.500000,1041.349976,962.012024,2484372
5,2023-01-03 00:00:00+00:00,INFY.NS,1519.849976,1532.650024,1515.099976,1522.550049,1458.243774,4551130
6,2023-01-03 00:00:00+00:00,TCS.NS,3252.000000,3320.000000,3245.300049,3311.350098,3208.393311,1245178
7,2023-01-03 00:00:00+00:00,WIPRO.NS,393.049988,397.750000,391.700012,397.049988,395.231537,2997211
8,2023-01-04 00:00:00+00:00,HCLTECH.NS,1036.199951,1042.000000,1031.500000,1036.599976,957.623901,1219170
9,2023-01-04 00:00:00+00:00,INFY.NS,1521.000000,1521.000000,1491.099976,1494.800049,1431.665771,7059590


In [ ]:
# Descriptive Statistics for each stock
descriptive_stats = data.groupby('Ticker')['Close'].describe()

print(descriptive_stats)

            count         mean         std          min          25%  \
Ticker                                                                 
HCLTECH.NS  450.0  1363.748111  241.334405  1032.800049  1136.149994   
INFY.NS     450.0  1531.766444  188.706617  1223.400024  1410.587494   
TCS.NS      450.0  3697.428555  388.486741  3089.600098  3368.412537   
WIPRO.NS    450.0   446.053000   57.538871   356.200012   396.900002   

                    50%          75%          max  
Ticker                                             
HCLTECH.NS  1318.800049  1567.849976  1871.750000  
INFY.NS     1484.599976  1642.487488  1968.099976  
TCS.NS      3606.099976  3983.299927  4553.750000  
WIPRO.NS     430.975006   494.787506   573.200012  


In [ ]:
# Time Series Analysis
data['Date'] = pd.to_datetime(data['Date'])
pivot_data = data.pivot(index='Date', columns='Ticker', values='Close')

# Create a subplot
fig = make_subplots(rows=1, cols=1)

# Add traces for each stock ticker
for column in pivot_data.columns:
    fig.add_trace(
        go.Scatter(x=pivot_data.index, y=pivot_data[column], name=column),
        row=1, col=1
    )

# Update layout
fig.update_layout(
    title_text='Time Series of Closing Prices',
    xaxis_title='Date',
    yaxis_title='Closing Price',
    legend_title='Ticker',
    showlegend=True
)

# Show the plot
fig.show()

In [ ]:
# Volatility Analysis
volatility = pivot_data.std().sort_values(ascending=False)

fig = px.bar(volatility,
             x=volatility.index,
             y=volatility.values,
             labels={'y': 'Standard Deviation', 'x': 'Ticker'},
             title='Volatility of Closing Prices (Standard Deviation)')

# Show the figure
fig.show()

In [ ]:
# Correlation Analysis
correlation_matrix = pivot_data.corr()

fig = go.Figure(data=go.Heatmap(
                    z=correlation_matrix,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    colorscale='blues',
                    colorbar=dict(title='Correlation'),
                    ))

# Update layout
fig.update_layout(
    title='Correlation Matrix of Closing Prices',
    xaxis_title='Ticker',
    yaxis_title='Ticker'
)

# Show the figure
fig.show()

In [ ]:
# Calculating the percentage change in closing prices
percentage_change = ((pivot_data.iloc[-1] - pivot_data.iloc[0]) / pivot_data.iloc[0]) * 100

fig = px.bar(percentage_change,
             x=percentage_change.index,
             y=percentage_change.values,
             labels={'y': 'Percentage Change (%)', 'x': 'Ticker'},
             title='Percentage Change in Closing Prices')

# Show the plot
fig.show()

In [ ]:
daily_returns = pivot_data.pct_change().dropna()

# Recalculating average daily return and standard deviation (risk)
avg_daily_return = daily_returns.mean()
risk = daily_returns.std()

# Creating a DataFrame for plotting
risk_return_df = pd.DataFrame({'Risk': risk, 'Average Daily Return': avg_daily_return})

fig = go.Figure()

# Add scatter plot points
fig.add_trace(go.Scatter(
    x=risk_return_df['Risk'],
    y=risk_return_df['Average Daily Return'],
    mode='markers+text',
    text=risk_return_df.index,
    textposition="top center",
    marker=dict(size=10)
))

# Update layout
fig.update_layout(
    title='Risk vs. Return Analysis',
    xaxis_title='Risk (Standard Deviation)',
    yaxis_title='Average Daily Return',
    showlegend=False
)

# Show the plot
fig.show()